In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.animation as animation

In [12]:
class CRNN(tf.Module):
    def __init__(self, init_p, exp_data, name="CRNN"):
        super(CRNN, self).__init__(name=name)
        #初始参数设置
        self.lb = 1e-8
        self.n_epoch = 2500
        self.n_plot = 10
        self.grad_max = 1e2
        self.maxiters = 50000

        self.lr_max = 5e-3
        self.lr_min = 1e-5
        self.lr_decay = 0.2
        self.lr_decay_step = 500
        self.w_decay = 1e-8

        self.llb = self.lb
        self.p_cutoff = -1.0
        self.R = -1.0 / 8.314e-3

        self.l_exp = tf.range(1,15)
        self.n_exp = 14

        self.nr=8
        self.ns=4

        self.npara = self.nr * (self.ns + 4) + 1

        #工作目录设置
        
        #在这里完成p矩阵分解。
        self.p = init_p
        
        
        #在这里完成实验数据整理，存入几个tf.constant中。
        self.exp_data = exp_data

    @tf.function
    def load_data(self):
        

    @tf.function
    def p2vec(self):
        slope = self.p[-1] * 1.e1
        
        w_b = self.p[:self.nr] * (slope * 10.0)
        w_b = tf.clip_by_value(w_b, clip_value_min=0., clip_value_max=50.)
        
        # julia 与 tf的reshape函数实现不同
        w_out = tf.reshape(self.p[self.nr:self.nr*(self.ns+1)],shape=(self.nr,self.ns))
        w_out = tf.transpose(w_out)

        temp1 = tf.clip_by_value(w_out[0:1, :], -3., 0.)
        temp2 = tf.clip_by_value(tf.abs(w_out[-1:, :]), 0., 3.)
        w_out = tf.concat([temp1, w_out[1:-1], temp2], axis=0)

        # 此段省略
        # julia原文：
        # if p_cutoff > 0.0
        #     w_out[findall(abs.(w_out) .< p_cutoff)] .= 0.0
        # end

        temp3 = -1*(tf.reduce_sum(w_out[:-2,:], 0) + w_out[-1:, :])
        temp3 = tf.reshape(temp3, shape=(1,self.nr))
        
        w_out = tf.concat([w_out[:-2,:], temp3, w_out[-1:,:]], axis=0)

        w_in_Ea = tf.abs(self.p[self.nr*(self.ns+1):self.nr*(self.ns+2)]) * slope * 100.
        w_in_Ea = tf.clip_by_value(w_in_Ea, 0., 300.)
        w_in_Ea = tf.reshape(w_in_Ea, shape=(1,8))

        w_in_b = tf.abs(self.p[self.nr*(self.ns+2):self.nr*(self.ns+3)])
        w_in_b = tf.reshape(w_in_b, shape=(1,8))

        w_in_ocen = tf.abs(self.p[self.nr*(self.ns+3):self.nr*(self.ns+4)])
        w_in_ocen = tf.clip_by_value(w_in_ocen, 0., 1.5)
        w_in_ocen = tf.reshape(w_in_ocen, shape=(1,8))

        # 此段省略
        # julia原文：
        # if p_cutoff > 0:
        #     w_in_ocean[abs(w_in_ocean)<p_cutoff] = 0.0
        
        w_in = tf.concat([
            tf.clip_by_value(w_out*(-1),0.,4.),
            w_in_Ea,
            w_in_b,
            w_in_ocen
        ], 0)

        return w_in, w_b, w_out

    @tf.function
    def crnn(self, t, u):
        pass

    @tf.function
    def getsampletemp(self, t):
        pass

    @tf.function
    def pred_n_ode(self):
        self.w_in, self.w_b, self.w_out = self.p2vec()
        pass

In [13]:
p_test = tf.constant(np.loadtxt("p.txt"), dtype=tf.float32)
exp_data_test = np.array([1,2,3])
model = CRNN(p_test, exp_data_test)

In [23]:
p = tf.Variable(p_test)
model = CRNN(p, exp_data_test)
with tf.GradientTape() as tape:
    tape.watch(p)
    
    w_in = model.w_in
grad = tape.gradient(w_in, p)

In [25]:
grad